In [10]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow import keras

In [9]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [45]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [46]:
print(x_train.shape,y_train.shape)

(60000, 28, 28) (60000,)


In [47]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)
input_shape=(28,28,1)

In [48]:
# Convert binary to categorical

In [49]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test,10)

In [50]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [51]:
x_train/=255
x_test/=255

In [52]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [56]:
# Creating the model

batch_size = 128
num_classes = 10
epochs = 10

model = tf.keras.Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adadelta(),metrics=['accuracy'])

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [57]:
# Train the model

In [76]:
hist = model.fit(x_train, y_train,batch_size=32,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

model.save('mnist.h5')
print("Saving the model as mnist.h5")

Epoch 1/10
1875/1875 [==============================] - 46s 24ms/step - loss: 0.1248 - accuracy: 0.9611 - val_loss: 0.0545 - val_accuracy: 0.9827
Epoch 2/10
1875/1875 [==============================] - 46s 24ms/step - loss: 0.0450 - accuracy: 0.9858 - val_loss: 0.0398 - val_accuracy: 0.9871
Epoch 3/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0288 - accuracy: 0.9907 - val_loss: 0.0332 - val_accuracy: 0.9904
Epoch 4/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0208 - accuracy: 0.9937 - val_loss: 0.0342 - val_accuracy: 0.9884
Epoch 5/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0153 - accuracy: 0.9950 - val_loss: 0.0330 - val_accuracy: 0.9901
Epoch 6/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0115 - accuracy: 0.9963 - val_loss: 0.0367 - val_accuracy: 0.9900
Epoch 7/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.0093 - accuracy: 0.9970 - val_loss: 0.0299 -

In [77]:
from tensorflow.keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [78]:

model = load_model('mnist.h5')

In [79]:
def predict_img(img):
    img=img.resize((28,28))
    img=img.convert('L')
    img=np.array(img)
    img=img.reshape(1,28,28,1)
    img/=255.0
    
    res=model.predict([img])[0]
    return np.argmax(res),max(res)

In [81]:
model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()